# **Código para módulo SP32 Emisor**

In [ ]:
import network, time, socket, ujson, _thread, espnow, struct
from machine import Pin, ADC, SoftI2C
import ssd1306

# ----------------------------
# CONFIGURACIÓN
# ----------------------------
SSID = "CasaVDH2.4"        # mi WiFi personal
PASSWORD = "Villarreal"

PIN_ADC = 34            # AD8832 conectado a GPIO34
PIN_SCL = 26            # OLED SCL
PIN_SDA = 27            # OLED SDA
WIDTH = 128
HEIGHT = 32

# MAC del receptor
PEER_MAC = b"\x6c\xc8\x40\x92\xb5\x2c"

# ----------------------------
# INICIALIZAR ADC
# ----------------------------
adc = ADC(Pin(PIN_ADC))
adc.atten(ADC.ATTN_11DB)       # Rango 0–3.3V
adc.width(ADC.WIDTH_12BIT)     # Resolución 12 bits (0–4095)

# ----------------------------
# INICIALIZAR OLED
# ----------------------------
i2c = SoftI2C(scl=Pin(PIN_SCL), sda=Pin(PIN_SDA))
oled = ssd1306.SSD1306_I2C(WIDTH, HEIGHT, i2c)

def oled_show(ip):
    oled.fill(0)
    oled.text("EMG Tx ESP32", 0, 0)
    oled.text("IP:", 0, 12)
    oled.text(ip, 0, 24)
    oled.show()

# ----------------------------
# CONEXIÓN WIFI
# ----------------------------
def connect_wifi():
    sta = network.WLAN(network.STA_IF)
    sta.active(True)
    if not sta.isconnected():
        print("Conectando a WiFi...")
        sta.connect(SSID, PASSWORD)
        t0 = time.time()
        while not sta.isconnected():
            if time.time() - t0 > 15:
                break
            time.sleep(1)
    print("Estado WiFi:", sta.ifconfig())
    ip = sta.ifconfig()[0]
    oled_show(ip)
    return sta

sta = connect_wifi()

# ----------------------------
# SERVIDOR HTTP
# ----------------------------
def run_http():
    addr = socket.getaddrinfo("0.0.0.0", 80)[0][-1]
    s = socket.socket()
    s.bind(addr)
    s.listen(1)
    print("Servidor listo en http://{}:80/api/emg".format(sta.ifconfig()[0]))

    while True:
        cl, addr = s.accept()
        req = cl.recv(512)
        req_str = req.decode()

        if "GET /api/emg" in req_str:
            value = adc.read()
            response = ujson.dumps({"emg_value": value})
            cl.send("HTTP/1.1 200 OK\r\n")
            cl.send("Content-Type: application/json\r\n")
            cl.send("Access-Control-Allow-Origin: *\r\n\r\n")
            cl.send(response)
        else:
            cl.send("HTTP/1.1 404 Not Found\r\n\r\n")

        cl.close()

# ----------------------------
# ESPNOW EMISOR
# ----------------------------
def run_espnow():
    sta.active(True)
    e = espnow.ESPNow()
    e.active(True)

    try:
        e.add_peer(PEER_MAC)
    except:
        pass

    print("Enviando datos EMG al receptor:", PEER_MAC)

    while True:
        value = adc.read()
        packet = struct.pack(">H", value)  # 2 bytes
        e.send(PEER_MAC, packet)
        time.sleep_ms(100)

# ----------------------------
# MULTIHILO
# ----------------------------
def main():
    # Servidor HTTP en hilo secundario
    _thread.start_new_thread(run_http, ())
    # ESPNOW en hilo principal
    run_espnow()

main()